In [ ]:
# kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
competition_name = "jigsaw-toxic-comment-classification-challenge"

# Mount your Google Drive.
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#text cleaning
import regex as re
import string

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/projects/interpretability/kaggle_data/train.csv")
df.head()

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text.strip()

In [ ]:
df["clean_text"] = df.comment_text.apply(lambda x : clean_text(x))

In [ ]:
X = df.clean_text
y = df[classes].values

### Train Tokenizer


In [ ]:
from tokenizers import ByteLevelBPETokenizer

In [ ]:
vocab_size = 32000 #(hyper param)

In [ ]:
!mkdir /content/drive/MyDrive/projects/tokenizer/

In [ ]:
tokenizer_path = '/content/drive/MyDrive/your_path'


special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(
    iterator=X, vocab_size=vocab_size, min_frequency=2, show_progress=True, special_tokens=special_tokens
)
tokenizer.save_model(tokenizer_path)

Will take 1710 as max_seq_len

In [ ]:
X["encoded_ids"] = X.apply(lambda x: len(tokenizer_loaded.encode(x)) if type(x)== str else 0)
max(X["encoded_ids"].values) #1709

In [ ]:
import tempfile
import os


def load_tokenizer(load_path):
      # Load tokenizer from the temporary path
      tokenizer = ByteLevelBPETokenizer(load_path+ '/vocab.json', load_path + '/merges.txt')
      return tokenizer

In [ ]:
tokenizer_path = '/content/drive/MyDrive/projects/tokenizer/'
tokenizer_loaded = load_tokenizer(tokenizer_path)

In [ ]:
def encode(text, tokenizer, hparams):
  encoding = tokenizer.encode(text)
  encoding.truncate(max_length=hparams.max_seq_len)
  encoding.pad(length=hparams.max_seq_len)
  return encoding.ids


In [ ]:
input_ids = encode(X[0],tokenizer_loaded, hparams)

In [ ]:
def preprocess_sample(sample, tokenizer):
  input_ids = encode(sample,tokenizer,hparams)
  return input_ids